In [1]:
# UART to Avalon-MM bridge送受信

from serial import *
from struct import *

def print_bin(label, bin):
	if print_bin.status:
		print(label, end=" : ")
		for b in bin: print(format(b, "02x"), end=" ")
		print("")

print_bin.status = False	# Trueならバイナリプリントする 

# Packetデータのデコードとエンコード

def bytes_decode(byte):
	if byte == 0x7a:
		bytes_decode.status |= 1<<0
		bytes_decode.result = b""
		return
	elif byte == 0x7b:
		bytes_decode.status |= 1<<1
		return
	elif byte == 0x7c:
		bytes_decode.status |= 1<<2
		return
	elif byte == 0x7d:
		bytes_decode.status |= 1<<3
		return

	if (bytes_decode.status & (1<<2)) != 0:
		bytes_decode.status &= ~(1<<2)
		return

	if (bytes_decode.status & (1<<3)) != 0:
		bytes_decode.status &= ~(1<<3)
		byte ^= 0x20

	if (bytes_decode.status & (1<<0)) != 0:
		bytes_decode.result += pack("B", byte)

	if (bytes_decode.status & (1<<1)) != 0:
		bytes_decode.status = 0
		return bytes_decode.result

bytes_decode.status = 0
bytes_decode.result = b""

def bytes_encode(bin):
	bytes = b"\x7c\x00\x7a"
	for idx,c in enumerate(bin):
		if idx == len(bin) - 1:
			bytes += b"\x7b"

		if c >= 0x7a and c <= 0x7d:
			bytes += pack("BB", 0x7d, c ^ 0x20)
		else:
			bytes += pack("B", c)
	return bytes

# Avalon-MMのライト

def avm_writetransaction(ser, code, addr, wrbytes):
	send_data = bytes_encode(pack("!BxHL", code, len(wrbytes), addr) + wrbytes)
	s = "->  IOWR" if code == 0x00 else "-> MEMWR"
	print_bin(s, send_data)
	ser.write(send_data)
	res = None
	while res is None: res = bytes_decode(ser.read()[0])
	print_bin("<-   RES", res)
	resp, size = unpack("!BxH", res[0:4])
	return size

def avm_iowr(ser, addr, wrbytes):
	return avm_writetransaction(ser, 0x00, addr, wrbytes)

def avm_iowr8(ser, base, offset, wrdata):
	return avm_writetransaction(ser, 0x00, base+(offset*4), pack("B", wrdata))

def avm_iowr16(ser, base, offset, wrdata):
	return avm_writetransaction(ser, 0x00, base+(offset*4), pack("H", wrdata))

def avm_iowr32(ser, base, offset, wrdata):
	return avm_writetransaction(ser, 0x00, base+(offset*4), pack("L", wrdata))

def avm_memwr(ser, addr, wrbytes):
	return avm_writetransaction(ser, 0x04, addr, wrbytes)

# Avalon-MMのリード

def avm_readtransaction(ser, code, addr, readnum):
	send_data = bytes_encode(pack("!BxHL", code, readnum, addr))
	s = "->  IORD" if code == 0x10 else "-> MEMRD"
	print_bin(s, send_data)
	ser.write(send_data)
	res = None
	while res is None: res = bytes_decode(ser.read()[0])
	print_bin("<-   RES", res)
	return res

def avm_iord(ser, addr, readnum):
	return avm_readtransaction(ser, 0x10, addr, readnum)

def avm_iord8(ser, base, offset):
	res = avm_readtransaction(ser, 0x10, base+(offset*4), 1)
	return unpack("B", res)[0]

def avm_iord16(ser, base, offset):
	res = avm_readtransaction(ser, 0x10, base+(offset*4), 2)
	return unpack("H", res)[0]

def avm_iord32(ser, base, offset):
	res = avm_readtransaction(ser, 0x10, base+(offset*4), 4)
	return unpack("L", res)[0]

def avm_memrd(ser, addr, readnum):
	return avm_readtransaction(ser, 0x14, addr, readnum)


In [2]:
# c4e_dvp_coreへのアクセス 

from datetime import *

SERIAL_PORT = "COM4" 
SERIAL_BITRATE = 230400

SDRAM_BASE	= 0x00000000
SYSID_BASE	= 0x10000000
LED_BASE	= 0x10000040
VGA_BASE	= 0x10000100
CAM_BASE	= 0x10000200

CAM_CONTINUOUS	= (1<<2)
CAM_SINGLE		= (0<<2)
CAM_READY		= (1<<1)
CAM_START		= (1<<0)

import ov5642_sccb_fx3
OV5642_REGLIST = ov5642_sccb_fx3.reg_list

with Serial(SERIAL_PORT, SERIAL_BITRATE) as ser:
	ser.reset_input_buffer()
	print_bin.status = False

	# 初期化
	avm_iowr32(ser, LED_BASE, 0, 0)
	avm_iowr32(ser, VGA_BASE, 0, 0)
	avm_iowr32(ser, CAM_BASE, 0, 0)
	_i = True
	while _i:
		_i = (avm_iord32(ser, CAM_BASE, 0) & CAM_READY == 0)
	print("CAM Peripheral initial done.")

	# SystemID読み出し
	sysid = avm_iord32(ser, SYSID_BASE, 0)
	timestamp = avm_iord32(ser, SYSID_BASE, 1)
	dt = datetime.fromtimestamp(timestamp)
	print("System ID =", hex(sysid), ", Build", dt)

	# OV5642の初期化
	for regdata in OV5642_REGLIST:
		avm_iowr32(ser, CAM_BASE, 3, regdata)

	print("SCCB write success.")

	# DVPキャプチャ開始 
	avm_iowr32(ser, CAM_BASE, 1, int(1280*2*720/4))		# REG1:キャプチャデータ数(4バイト単位)
	avm_iowr32(ser, CAM_BASE, 2, SDRAM_BASE)			# REG2:ストア先
	avm_iowr32(ser, CAM_BASE, 0, CAM_CONTINUOUS | CAM_START) # 連続キャプチャ開始

	# VGA表示開始 (1280x720から640x480を切り出し)
#	avm_iowr32(ser, VGA_BASE, 1, SDRAM_BASE + int((1280-640) + (720-480)*1280)) # REG1:表示開始先頭アドレス
	avm_iowr32(ser, VGA_BASE, 1, SDRAM_BASE + int((1280-800) + (720-600)*1280)) # REG1:表示開始先頭アドレス
	avm_iowr32(ser, VGA_BASE, 0, 1)

	print("Capture start.")

	# LED点灯  
	avm_iowr32(ser, LED_BASE, 0, 1)


CAM Peripheral initial done.
System ID = 0xa0230111 , Build 2023-01-11 04:11:59
SCCB write success.
Capture start.
